___

### les bases
___
* sélectionner un estimateur et préciser ses hyperparamètres : modele = LinearRegression(.........)
* Entrainer le modèle sur les données X, y (divisées en 2 tableaux Numpy) : model.fit(X, y)
* Evaluer le modèle : model.score(X, y)
* Utiliser le modèle : model.predict(X)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
# df_original = pd.read_csv("données d'origine.csv")
df_original = pd.read_csv("data_cleaned_cheated.csv")
df = df_original[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges', 'charges_group_cheat']]

df['group'] = np.where(df['charges'] < 4746.344000, '1', 
                             np.where(df['charges'] < 9386.161300, '2', 
                             np.where(df['charges'] < 16657.717450, '3', '4')))

# df = df[df['charges'] < 49000]
# df_group['group'] = np.where(df_group['charges'] < 4719.736550, '1', 
                            #  np.where(df_group['charges'] < 9301.893550, '2', 
                            #  np.where(df_group['charges'] < 16297.846000, '3', '4')))

mapping_sex = {'male': 0, 'female': 1}
mapping_smoker = {'yes': 1, 'no': 0}

# df['sex'] = df['sex'].map(mapping_sex)
# df['smoker'] = df['smoker'].map(mapping_smoker)
df['northeast'] = 0
df['southeast'] = 0
df['northwest'] = 0
df['southwest'] = 0
df.loc[df['region'] == 'northeast', 'northeast'] = 1
df.loc[df['region'] == 'southeast', 'southeast'] = 1
df.loc[df['region'] == 'northwest', 'northwest'] = 1
df.loc[df['region'] == 'southwest', 'southwest'] = 1

df.loc[:, 'smoker_bmi'] = df['smoker'] * df['bmi']
df.loc[:, 'smoker_age'] = df['smoker'] * df['age']

# df.loc[:, 'charges_group'] = np.where(df['smoker'] == 0, 1,
#                                       np.where(df['bmi'] >= 30, 3,
#                                                np.where(df['sex'] == 0, 2,
#                                                         np.where(df['age'] < 22, 1, 3))))
df.loc[:, 'charges_group'] = np.where(df['smoker'] == 0, 1,
                                      np.where(df['bmi'] >= 30, 3,
                                               2))

df.loc[:, 'diff_charges_group'] = abs(df['charges_group'] - df['charges_group_cheat'])
df['children'] = np.where(df['children'] > 0, 1, 0) 

# nettoyage
df = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'smoker_age', 'northeast', 'southeast', 'northwest', 'southwest', 'charges', 'charges_group_cheat', 'diff_charges_group', 'charges_group', 'group']]
# df = df[['age', 'sex', 'bmi', 'smoker', 'charges', 'charges_group', 'charges_group2', 'charges_group3']]

display(df, df['diff_charges_group'].sum())
# display(df.isnull().sum())

/tmp/ipykernel_12814/689199495.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['group'] = np.where(df['charges'] < 4746.344000, '1',


,age,sex,bmi,children,smoker,smoker_bmi,smoker_age,northeast,southeast,northwest,southwest,charges,charges_group_cheat,diff_charges_group,charges_group,group
0,19,1,27.900,0,1,27.90,19,0,0,0,1,16884.92400,2,0,2,4
1,18,0,33.770,1,0,0.00,0,0,1,0,0,1725.55230,1,0,1,1
2,28,0,33.000,1,0,0.00,0,0,1,0,0,4449.46200,1,0,1,1
3,33,0,22.705,0,0,0.00,0,0,0,1,0,21984.47061,2,1,1,4
4,32,0,28.880,0,0,0.00,0,0,0,1,0,3866.85520,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,50,0,30.970,1,0,0.00,0,0,0,1,0,10600.54830,1,0,1,3
1333,18,1,31.920,0,0,0.00,0,1,0,0,0,2205.98080,1,0,1,1
1334,18,1,36.850,0,0,0.00,0,0,1,0,0,1629.83350,1,0,1,1
1335,21,1,25.800,0,0,0.00,0,0,0,0,1,2007.94500,1,0,1,1


106

___

### LinearRegression
___

In [4]:
from sklearn.model_selection import train_test_split # pour diviser le jeu de données initial en 1 jeu d'entrainement (X_train, y_train) et un jeu de test (X_test, y_test)
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

y = df[['charges']]
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'northeast', 'southeast', 'northwest', 'southwest']] # southeast
X = df[['age', 'bmi', 'children', 'smoker', 'smoker_bmi', 'charges_group']]
X_dummies = pd.get_dummies(df[['age', 'bmi', 'smoker', 'smoker_bmi', 'children']], columns=['age', 'bmi', 'smoker', 'smoker_bmi', 'children'], drop_first=True)
# display(X)

X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, shuffle=True, train_size=0.85, random_state=42, stratify=X_dummies['smoker_1'])
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_train)
predictions2 = model.predict(X_test)
# print(model.score(X_train, y_train), r2_score(y_train, predictions))
print(model.score(X_test, y_test), r2_score(y_test, predictions2))

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(round(model.score(X_test, y_test)*100, 2), "|", round(r2_score(y_test, predictions)*100, 2))

# model = LinearRegression()
# model.fit(X_train, y_train)
# model.score(X_train, y_train)

# step = np.arange(0,1.1,1)
# param_grid = {'alpha': np.arange(0,1),
#               'l1_ratio': np.arange(0,1)}
# grid = GridSearchCV(ElasticNet(), param_grid, cv = 42)
# grid.fit(X_train, y_train)

# print('')
# print('_______________________________________________________')
# print('###BEST### >>>', round(grid.best_score_, 2))

-2.1926967986367145e+26 -2.1926967986367145e+26
92.87 | 92.87


___

### PolynomialFeatures
___

In [5]:
from sklearn.model_selection import train_test_split # pour diviser le jeu de données initial en 1 jeu d'entrainement (X_train, y_train) et un jeu de test (X_test, y_test)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import RobustScaler

y = df[['charges']]
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'northeast', 'southeast', 'northwest', 'southwest']] # southeast
X = df[['age', 'bmi', 'children', 'smoker', 'smoker_bmi', 'charges_group']]
# display(X)

# X = pd.get_dummies(X[['age', 'bmi', 'children', 'smoker', 'smoker_bmi', 'charges_group']], columns=['age', 'smoker'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

X_train_robust = RobustScaler().fit_transform(X_train)

#creer variables polynomiales a partir de nos variables
X_poly = PolynomialFeatures(2).fit_transform(X_train_robust)
X_poly_robust = RobustScaler().fit_transform(X_poly)

# Crée le modèle de régression linéaire
model = LinearRegression()

# Entraîne le modèle sur les données d'entraînement
model.fit(X_poly, y_train)

# Faire des prédictions sur les données d'entraînement
predictions_train = model.predict(X_poly_robust)

# Calcule le R^2 et l'erreur quadratique moyenne (MSE) sur les données d'entraînement
r2_train = r2_score(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
rmse_train = np.sqrt(mse_train)

print(f"R^2 Score (Train): {r2_train}")
# print(f"MSE (Train): {mse_train}")
# print(f"RMSE (Train): {rmse_train}")

R^2 Score (Train): 0.8515265431219851


In [6]:
from sklearn.model_selection import train_test_split # pour diviser le jeu de données initial en 1 jeu d'entrainement (X_train, y_train) et un jeu de test (X_test, y_test)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import RobustScaler

y = df[['charges']]
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'smoker_bmi', 'northeast', 'southeast', 'northwest', 'southwest']] # southeast
X = df[['age', 'bmi', 'children', 'smoker', 'smoker_bmi']]

# X = pd.get_dummies(X[['age', 'bmi', 'children', 'smoker', 'smoker_bmi', 'charges_group']], columns=['smoker'], drop_first=True)
# display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

X_train_robust = RobustScaler().fit_transform(X_train)
X_test_robust = RobustScaler().fit_transform(X_test)

#creer variables polynomiales a partir de nos variables
PolyNome = 2
X_poly = PolynomialFeatures(PolyNome).fit_transform(X_train_robust)
X_poly_robust = RobustScaler().fit_transform(X_poly)
X_test_poly = PolynomialFeatures(PolyNome).fit_transform(X_test_robust)

# Crée le modèle de régression linéaire
model = LinearRegression()

# Entraîne le modèle sur les données d'entraînement
model.fit(X_poly, y_train)

# Faire des prédictions sur les données d'entraînement
predictions_train = model.predict(X_poly_robust)

# Calcule le R^2 et l'erreur quadratique moyenne (MSE) sur les données d'entraînement
# r2_train = r2_score(y_train, predictions_train)
# mse_train = mean_squared_error(y_train, predictions_train)
# rmse_train = np.sqrt(mse_train)

# print(f"R^2 Score (Train): {r2_train}")

model = LinearRegression()
model.fit(X_poly, y_train)
print(round(model.score(X_test_poly, y_test)*100, 2))

90.38


___

MODELE AVEC PIPELINES
___

In [7]:
from sklearn.pipeline import make_pipeline

y = df[['charges']]
X = df[['age', 'bmi', 'children', 'smoker', 'smoker_bmi', 'charges_group']]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

PolyNome = 2
model = make_pipeline(
    RobustScaler(),
    PolynomialFeatures(PolyNome),
    LinearRegression()
)

model.fit(X_train, y_train)

predictions_train = model.predict(X_train)

r2_train = r2_score(y_train, predictions_train)
print(f"R^2 Score (Train): {r2_train}")

predictions_test = model.predict(X_test)

r2_test = r2_score(y_test, predictions_test)
print(f"R^2 Score (Test): {r2_test}")

R^2 Score (Train): 0.8578588884094903
R^2 Score (Test): 0.9260398829084634
